# Intro
In this getting started tutorial we want to setup authentication

# Table of Contents
1. [Introduction](#first-bullet)
2. [Accessing Market Data](#second-bullet)
3. [Backtesting](#third-bullet)
4. [Paper Trading](#fourth-bullet)

# Introduction <a class="anchor" id="first-bullet"></a>

To follow along, you'll need to use your Alpaca API paper keys.

If you don't have keys, sign up here:
[Free Stock and Crypto Data with $0 Commission Trading](https://app.alpaca.markets/signup)

You can find your keys on your [dashboard](https://app.alpaca.markets/paper/dashboard/overview). If you're having trouble finding your keys, follow [this tutorial](https://alpaca.markets/learn/connect-to-alpaca-api/). 

In [1]:
# API Credentials
import os
API_KEY = os.getenv("APCA_API_KEY_ID")
SECRET_KEY =  os.getenv("APCA_API_SECRET_KEY")

Now we can get started! We will need to import `alpaca_trade_api`. 

If we haven't installed it yet, we can do so by running `pip install alpaca_trade_api`.

Then let's define an instance of the REST API interface and provide it our API keys.

In [2]:
%%capture
!pip install --upgrade pip
!pip install alpaca-py==0.17.0
!pip install backtrader
!pip install PyYAML==6.0.1
!pip install matplotlib==3.2.2
!pip install plotly

In [16]:
from alpaca.data.historical import CryptoHistoricalDataClient
from alpaca.data.requests import CryptoBarsRequest
from alpaca.data.timeframe import TimeFrame
from alpaca.trading.client import TradingClient
from alpaca.trading.requests import MarketOrderRequest
from alpaca.trading.enums import OrderSide, TimeInForce
from datetime import datetime
from dateutil.relativedelta import relativedelta
import logging

paper = True
api = TradingClient(
    os.getenv("APCA_API_KEY_ID"), os.getenv("APCA_API_SECRET_KEY"), paper=paper
)

logging.basicConfig(
    level=logging.INFO, format="%(asctime)s - %(name)s - %(levelname)s - %(message)s"
)
logger = logging.getLogger(__name__)


# Define the secrets that are

# Accessing Market Data  <a class="anchor" id="second-bullet"></a>

The REST API allows you to access historical market data, account data, and place trades.

Here are some of the types of the market data Alpaca offers:
*   Bar data is available in custom timeframes (1 day, 30 mins, 5 mins, etc)
*   Quote and Trade data is also available

Let's take a look at daily bar data for Bitcoin during the year of 2021.

In [37]:
# Retrieve daily bar data for Bitcoin in a DataFrame 
          # Alpaca Market Data Client
data_client = CryptoHistoricalDataClient()
trading_pair = "BTC/USD"
time_diff = datetime.now() - relativedelta(hours=1000)
logger.info(
    "Getting bar data for {0} starting from {1}".format(trading_pair, time_diff)
)
# Defining Bar data request parameters
request_params = CryptoBarsRequest(
    symbol_or_symbols=[trading_pair], timeframe=TimeFrame.Hour, start=time_diff
)

btc_bars = data_client.get_crypto_bars(request_params).df
print(btc_bars)
#btc_quotes = data_client.get_crypto_latest_quote(request_params).df
#btc_trades = data_client.get_crypto_trades(request_params).df

2024-05-01 02:21:07,415 - __main__ - INFO - Getting bar data for BTC/USD starting from 2024-03-20 10:21:07.415862


                                        open        high         low  \
symbol  timestamp                                                      
BTC/USD 2024-03-20 11:00:00+00:00  63006.320  63919.7930  62951.8265   
        2024-03-20 12:00:00+00:00  63328.760  64292.6310  63200.3605   
        2024-03-20 13:00:00+00:00  64011.350  64011.3500  63066.2315   
        2024-03-20 14:00:00+00:00  63551.435  64328.0895  63442.7230   
        2024-03-20 15:00:00+00:00  64250.524  64366.1450  63447.9100   
...                                      ...         ...         ...   
        2024-04-30 22:00:00+00:00  60137.100  60959.0600  60093.4535   
        2024-04-30 23:00:00+00:00  60502.170  60872.3050  60377.9825   
        2024-05-01 00:00:00+00:00  60634.715  60757.3500  60095.4140   
        2024-05-01 01:00:00+00:00  60142.359  60287.6605  59820.0550   
        2024-05-01 02:00:00+00:00  60058.970  60058.9700  59581.7450   

                                        close    volume  trade_

Using Jupyter notebooks allow you to easily visualize data and test ideas. 

For example, we can use plotly to make beautiful candle charts of Bitcoins's daily bar data. 

We can also calculate the 13 day simple moving average (SMA) for Bitcoin over that period using pandas operations, and add it onto our chart.

In [45]:
# plotly imports
import plotly.graph_objects as go
import plotly.express as px

btc_bars = btc_bars.reset_index()

# SPY bar data candlestick plot
candlestick_fig = go.Figure(data=[go.Candlestick(x=btc_bars.index,
                open=btc_bars['open'],
                high=btc_bars['high'],
                low=btc_bars['low'],
                close=btc_bars['close'])])

# Calculating 13 day SMA using pandas rolling mean
sma = btc_bars['close'].rolling(13).mean().dropna()

# Creating a line plot for our sma
sma_fig = px.line(x=sma.index, y=sma)

# Adding both plots onto one chart
fig = go.Figure(data=candlestick_fig.data + sma_fig.data)

# Adding a title and axes labels
fig.update_layout(
    title=f"Price of Bitcoin",
    xaxis_title="Date (days)",
    yaxis_title="Price ($USD)",
)

# Displaying our chart
fig.show()

#Backtesting  <a class="anchor" id="third-bullet"></a>

We can use backtrader along with Alpaca’s Market Data API to backtest a strategy’s performance with historical data. Backtrader allows you to focus on writing reusable trading strategies, indicators and analyzers instead of having to spend time building infrastructure. 

Below we've written a function that allows you to easily backtest a strategy written in backtrader. The function takes in as a parameter the strategy you wish to backtest, the symbols the strategy trades on, start and end dates of the backtest, the timeframe of the strategy, and the initial cash of the backtest. 


In [153]:
import backtrader as bt
import matplotlib as mpl
import pandas as pd

mpl.rcParams['figure.dpi'] = 150

def run_backtest(strategy, symbols, start, end, timeframe=TimeFrame.Day, cash=10000):
    """params:
       strategy: the strategy you wish to backtest, an instance of backtrader.Strategy
       symbols: the symbol (str) or list of symbols List[str] you wish to backtest on
       start: start date of backtest in format 'YYYY-MM-DD'
       end: end date of backtest in format: 'YYYY-MM-DD'
       timeframe: the timeframe the strategy trades on (size of bars) -
                   1 min: TimeFrame.Minute, 1 day: TimeFrame.Day, 5 min: TimeFrame(5, TimeFrameUnit.Minute)
       cash: the starting cash of backtest
 """
    start = pd.to_datetime(start).date()
    end = pd.to_datetime(end).date()
    #timeframe = pd.to_datetime(timeframe)
    request_params = CryptoBarsRequest(
      symbol_or_symbols=symbols, timeframe=timeframe, start=start, end=end
    )

    print(f'Running backtest for {strategy.__name__} on {symbols} from {start} to {end} with {timeframe} bars')
    # initialize backtrader broker
    cerebro = bt.Cerebro(stdstats=True)
    cerebro.broker.setcash(cash)

    # add strategy
    cerebro.addstrategy(strategy)

    # add analytics
    # cerebro.addobserver(bt.observers.Value)
    # cerebro.addobserver(bt.observers.BuySell)
    cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='mysharpe')
    # historical data request
    if type(symbols) == str:
        symbol = symbols
        alpaca_data = data_client.get_crypto_bars(request_params).df
        data = bt.feeds.PandasData(dataname=alpaca_data, name=symbol)
        cerebro.adddata(data)
    elif type(symbols) == list or type(symbols) == set:
        for symbol in symbols:
            alpaca_data = data_client.get_crypto_bars(request_params).df
            data = bt.feeds.PandasData(dataname=alpaca_data, name=symbol)
            cerebro.adddata(data)

    # run
    initial_portfolio_value = cerebro.broker.getvalue()
    print(f'Starting Portfolio Value: {initial_portfolio_value}')
    results = cerebro.run()
    # final_portfolio_value = cerebro.broker.getvalue()
    # print(
    #     f"Final Portfolio Value: {final_portfolio_value} ---> Return: {(final_portfolio_value/initial_portfolio_value - 1)*100}%"
    # )

    # strat = results[0]
    # print("Sharpe Ratio:", strat.analyzers.mysharpe.get_analysis()["sharperatio"])
    # cerebro.plot(iplot=False)

## Backtesting Example 1: Simple Moving Average Crossover

Below is an example of a simple backtrader strategy. It is a simple moving average crossover strategy which trades when the 13 day SMA crosses over the 25 day SMA, and sells when it crosses under.  In this example, a crossover indicator is created which is 1 when the fast SMA crosses over the slow SMA, -1 when the fast crosses below the slow, and 0 if there is no crossover.

Take note of the structure of the strategy. We can define parameters for the strategy in the `params` dictionary. Any indicators can be defined in `__init__`. You can find a full list of indicators in the backtrader documentation. The `next()` method is called each iteration of the backtest. 


At the end you can run `run_backtest(SmaCross, 'BTCUSD', '2019-01-01', '2021-11-01', TimeFrame.Day, 10000)`, which will compute the backtest and output the results.



In [154]:
class SmaCross(bt.Strategy):
  
  # list of parameters which are configurable for the strategy
    params = dict(
        pfast=13,  # period for the fast moving average
        pslow=25   # period for the slow moving average
    )

    def __init__(self):
        sma1 = bt.ind.SMA(period=self.p.pfast)  # fast moving average
        sma2 = bt.ind.SMA(period=self.p.pslow)  # slow moving average
        self.crossover = bt.ind.CrossOver(sma1, sma2)  # crossover signal
  
    def next(self):
        if not self.position and self.crossover > 0:  # not in the market
            self.buy()
           
        elif self.position and self.crossover < 0:  # in the market & cross to the downside
            self.close()  # close long position


run_backtest(SmaCross, 'BTC/USD', '2023-01-01', '2024-01-01', TimeFrame.Day, 10000)

Running backtest for SmaCross on BTC/USD from 2023-01-01 to 2024-01-01 with 1Day bars
Starting Portfolio Value: 10000


AttributeError: 'tuple' object has no attribute 'to_pydatetime'

## Backtesting Example 2: Crypto Portfolio

Another class of strategies are rebalancing strategies. These strategies attempt to match a desired portfolio. Some rebalancing strategies change the securities in their portfolio based on market conditions, while others maintain a steady allocation through all conditions.

We'll create our own rebalancing strategy by creating a portfolio made from a subset of the most valuable coins available. This strategy aims to keep the same allocation of assets throughout our entire backtest period.

We'll create the portfolio by allocating to these cryptocurrencies:
* 35% Bitcoin (BTCUSD)
* 35% Ethereum (ETHUSD)
* 15% Solana (SOLUSD)
* 7.5% Dogecoin (DOGEUSD)
* 7.5% Litecoin (LTCUSD)


In [149]:
class CryptoRebalance(bt.Strategy):

   def __init__(self):
       # the last year we rebalanced (initialized to -1)
       self.year_last_rebalanced = -1 
       self.weights = { "BTC/USD" : 0.35 , "ETH/USD" : 0.35, "SOL/USD": 0.15, "DOGE/USD" : 0.075, "LTC/USD" : 0.075 }

   def next(self):
       # if we’ve already rebalanced this year
       if self.datetime.date().year == self.year_last_rebalanced:
           return
       # update year last balanced
       self.year_last_rebalanced = self.datetime.date().year
       # enumerate through each security
       for i,d in enumerate(self.datas):
           # rebalance portfolio with desired target percents
           symbol = d._name
           self.order_target_percent(d, target=self.weights[symbol])

        
symbols = ["BTC/USD", "ETH/USD", "SOL/USD", "DOGE/USD", "LTC/USD"]
run_backtest(CryptoRebalance, symbols , '2023-01-01', '2024-04-30', TimeFrame.Day, 10000)

Running backtest for CryptoRebalance on ['BTC/USD', 'ETH/USD', 'SOL/USD', 'DOGE/USD', 'LTC/USD'] from 2023-01-01 to 2024-04-30 with 1Day bars
Starting Portfolio Value: 10000


AttributeError: 'tuple' object has no attribute 'to_pydatetime'

# Paper Trading  <a class="anchor" id="fourth-bullet"></a>

We can use Alpaca’s trade API to place paper trades on our Alpaca account. Paper trading is free and doesn’t require you to fund your account. Let’s place trades so that we can create the example crypto portfolio in our paper account.





In [68]:
import random 

percent_allocations = { "BTCUSD" : 0.35 , "ETHUSD" : 0.35, "SOLUSD": 0.15, "DOGEUSD" : 0.075, "LTCUSD" : 0.075 }
max_qty_precision = {"BTCUSD" : 4 , "ETHUSD" : 3, "SOLUSD": 2, "DOGEUSD" : 0, "LTCUSD" : 2 }

# liquidate all existing positions before rebalancing
#rest_api.close_all_positions()

# get available cash
available_cash = float(rest_api.get_account().cash)

# how many dollars we want to allocate to each symbol
dollar_value_allocations = {symbol: percent * available_cash for symbol, percent in percent_allocations.items()}

# Rebalance portfolio
for symbol, dollars_alloc in dollar_value_allocations.items():
     
    # market price of current ETF
    market_price = rest_api.get_latest_crypto_bar(symbol, exchange="FTXU").close
    
    # how many shares we want, rounded to the most allowed decimal places
    target_holdings = round(dollars_alloc / market_price, max_qty_precision[symbol])
     
    # how many shares we have to buy to match target
    order_quantity = target_holdings
     
    # submit market order for this ETF
    print(f"Submitting market order for {order_quantity} units of {symbol}")

    #rest_api.submit_order(symbol, order_quantity, 'buy', 'market', client_order_id=f'cryptoc_{random.randrange(10000000)}')

NameError: name 'rest_api' is not defined